In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
hyperparameters = {
    'batch_size': 8,
    'num_workers': 16,
    'vocab_size': 91,
    'cnn_channels': [3, 64, 128, 256, 512],
    'hidden_dim': 512,
    'activation': 'silu',
    'num_heads': 8,
    'num_decoder_layers': 6,
    'dim_feedforward': 1024,
    'dropout': 0.1,
}

In [ ]:
from data import MathEquationsDatamodule
data_modules = []

for i in range(1, 11):
    data_module = MathEquationsDatamodule(
        f'data/batch_{i}',
        hyperparameters['batch_size'],
        hyperparameters['num_workers'],
    )
    
    data_modules.append(data_module)

In [ ]:
from model import Image2LaTeXTransformer
from lightning.pytorch.utilities.model_summary import ModelSummary

model = Image2LaTeXTransformer(hyperparameters)
ModelSummary(model, max_depth=2)